In [6]:
import pandas as pd
import networkx as nx
import os


In [8]:


print(f"현재 작업 폴더: {os.getcwd()}")
print("CSV 파일 로드를 시도합니다...")

# --- 1. 데이터 로드 ---
try:
    csv_file_name = 'C:\\socialnetwork\\SocialNetwork_7\\bipartite_skill_long.csv'
    df = pd.read_csv(csv_file_name)
    print("\n--- CSV 데이터 샘플 ---")
    print(df.head())
    
except FileNotFoundError:
    print(f"오류: '{csv_file_name}' 파일을 찾을 수 없습니다.")
    exit()
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")
    exit()

# --- 2. 노드 ID 및 순서 정의 ---

# 두 종류의 노드 리스트를 생성 (이 순서가 파일 순서가 됩니다)
# .unique()는 중복을 제거하고 고유한 값만 남깁니다.
companies = list(df['기업명'].unique())
skills = list(df['Skill'].unique())

# [기업] 리스트 뒤에 [스킬] 리스트를 붙여 전체 노드 리스트 생성
all_nodes = companies + skills
total_vertices = len(all_nodes)

# Pajek ID는 1부터 시작하므로, 1-based ID를 매핑하는 딕셔너리 생성
# 예: {'우아한형제들': 1, '컬리': 2, ..., 'EMR': 150, 'BigQuery': 151, ...}
node_to_id = {node: i + 1 for i, node in enumerate(all_nodes)}

print(f"\n--- 노드 리스트 생성 완료 ---")
print(f"총 노드 수: {total_vertices} (기업: {len(companies)}, 스킬: {len(skills)})")


# --- 3. .net 파일 작성 (네트워크 구조) ---
net_file_name = 'company_skill.net'
try:
    with open(net_file_name, 'w', encoding='utf-8') as f:
        
        # 3-1. Vertices (노드) 섹션 작성
        f.write(f"*vertices {total_vertices}\n")
        
        # all_nodes 리스트 순서대로 노드 ID와 이름을 쓴다
        for i, node in enumerate(all_nodes):
            pajek_id = i + 1
            # 노드 이름에 띄어쓰기 등이 있어도 안전하도록 항상 따옴표로 감싼다
            f.write(f'{pajek_id} "{node}"\n')
            
        # 3-2. Edges (엣지) 섹션 작성
        f.write("*edges\n")
        
        # 원본 CSV(df)를 한 줄씩 읽으면서 엣지를 쓴다
        for index, row in df.iterrows():
            company_name = row['기업명']
            skill_name = row['Skill']
            
            # 딕셔너리에서 각 노드의 Pajek ID를 찾는다
            company_id = node_to_id[company_name]
            skill_id = node_to_id[skill_name]
            
            # [ID 1] [ID 2] [가중치]
            f.write(f"{company_id} {skill_id} 1\n") # 가중치는 1로 통일
            
    print(f"'{net_file_name}' (구조) 파일 생성 성공!")

except Exception as e:
    print(f".net 파일 작성 중 오류: {e}")


# --- 4. .clu 파일 작성 (파티션 구분) ---
clu_file_name = 'company_skill.clu'
try:
    with open(clu_file_name, 'w', encoding='utf-8') as f:
        
        # 4-1. Vertices (노드) 헤더 작성
        f.write(f"*vertices {total_vertices}\n")
        
        # 4-2. 파티션 ID 작성
        # .net 파일과 *정확히 같은 순서*로 파티션 ID를 쓴다
        
        # 기업 노드 (companies 리스트) 만큼 파티션 '1'을 쓴다
        for _ in companies:
            f.write("1\n") # 1번 그룹 = 기업
            
        # 스킬 노드 (skills 리스트) 만큼 파티션 '2'를 쓴다
        for _ in skills:
            f.write("2\n") # 2번 그룹 = 스킬
            
    print(f"'{clu_file_name}' (구분) 파일 생성 성공!")

except Exception as e:
    print(f".clu 파일 작성 중 오류: {e}")

현재 작업 폴더: c:\socialnetwork\SocialNetwork_7
CSV 파일 로드를 시도합니다...

--- CSV 데이터 샘플 ---
             기업명     Skill
0  우아한형제들(배달의민족)       EMR
1  우아한형제들(배달의민족)  BigQuery
2  우아한형제들(배달의민족)  Planning
3  우아한형제들(배달의민족)  Superset
4  우아한형제들(배달의민족)       AWS

--- 노드 리스트 생성 완료 ---
총 노드 수: 2039 (기업: 1260, 스킬: 779)
'company_skill.net' (구조) 파일 생성 성공!
'company_skill.clu' (구분) 파일 생성 성공!


In [9]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

print(f"현재 작업 폴더: {os.getcwd()}")
print("CSV 파일 로드를 시도합니다...")

# --- 0. 한글 폰트 설정 (Matplotlib) ---
# (중요) networkx 시각화 시 한글 라벨이 깨지지 않도록 폰트를 설정합니다.
# VM 환경에 'Nanum' 계열 폰트가 설치되어 있다고 가정합니다.
try:
    font_path = None
    # 시스템 폰트에서 'Nanum' 또는 'AppleGothic' (Mac) 'Malgun' (Windows) 찾기
    for font in fm.fontManager.ttflist:
        if 'Nanum' in font.name or 'AppleGothic' in font.name or 'Malgun' in font.name:
            font_path = font.path
            break
    
    if font_path:
        font_name = fm.FontProperties(fname=font_path).get_name()
        plt.rcParams['font.family'] = font_name
        print(f"한글 폰트 '{font_name}'을(를) 설정했습니다.")
    else:
        # VM의 기본 fallback 시도
        plt.rcParams['font.family'] = 'NanumBarunGothic'
        print("기본 'NanumBarunGothic' 폰트를 설정합니다.")
        
    plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

except Exception as e:
    print(f"한글 폰트 설정 중 오류 발생 (라벨이 깨질 수 있음): {e}")


# --- 1. 데이터 로드 ---
try:
    csv_file_name = 'C:\\socialnetwork\\SocialNetwork_7\\bipartite_skill_long.csv'
    df = pd.read_csv(csv_file_name)
    print("\n--- CSV 데이터 샘플 ---")
    print(df.head())
    
except FileNotFoundError:
    print(f"오류: '{csv_file_name}' 파일을 찾을 수 없습니다.")
    exit()

# --- 2. 이분 그래프 생성 (NetworkX) ---
B = nx.Graph()

# 노드 추가 (bipartite 속성으로 기업/스킬 구분)
companies = df['기업명'].unique()
skills = df['Skill'].unique()
B.add_nodes_from(companies, bipartite=0) # 0번 그룹 = 기업
B.add_nodes_from(skills, bipartite=1)   # 1번 그룹 = 스킬

# 엣지 추가
B.add_edges_from(df.values)

print(f"\n--- 네트워크 생성 완료 ---")
print(f"총 노드 수: {B.number_of_nodes()} (기업: {len(companies)}, 스킬: {len(skills)})")
print(f"총 엣지 수: {B.number_of_edges()}")

# --- 3. (핵심) 시각화를 위한 레이아웃 및 색상 설정 ---
# 3-1. 노드 집합 분리
company_nodes = {n for n, d in B.nodes(data=True) if d['bipartite'] == 0}
skill_nodes = set(B) - company_nodes

# 3-2. 레이아웃 설정: nx.bipartite_layout
# (중요) 이 함수는 두 그룹을 위/아래로 나눠 배치해 줍니다.
pos = nx.bipartite_layout(B, company_nodes) 

# 3-3. 색상 설정
color_map = ['skyblue' if node in company_nodes else 'lightgreen' for node in B]


# --- 4. 시각화 1: 전체 그래프 (라벨 없이 구조만) ---
print("\n[시각화 1] 전체 그래프를 그립니다 (라벨 없음)...")
try:
    plt.figure(figsize=(20, 15)) # 그림 크기 지정
    nx.draw(B, pos, 
            node_color=color_map, 
            with_labels=False,  # 라벨 끄기 (너무 많음)
            node_size=20,       # 노드 크기 줄이기
            width=0.1,          # 엣지 두께 줄이기
            alpha=0.4)          # 투명도
    plt.title("기업-스킬 이분 그래프 (전체 구조)", fontsize=25)
    plt.savefig("bipartite_full_graph.png")
    print("'bipartite_full_graph.png' 파일로 저장되었습니다.")
    plt.close() # 메모리 해제

except Exception as e:
    print(f"전체 그래프 시각화 중 오류: {e}")


# --- 5. 시각화 2: 상위 기업 서브그래프 (라벨 포함) ---
print("\n[시각화 2] 상위 5개 기업 서브그래프를 그립니다 (라벨 포함)...")
try:
    # 5-1. 가장 많이 등장하는(연결이 많은) 기업 5개 선정
    top_companies = df['기업명'].value_counts().index[:5].tolist()
    
    # 5-2. 이 기업들과 연결된 스킬 노드들만 추출
    edges_of_top = df[df['기업명'].isin(top_companies)]
    skills_of_top = edges_of_top['Skill'].unique()

    # 5-3. 서브그래프 생성
    sub_nodes = top_companies + list(skills_of_top)
    S = B.subgraph(sub_nodes)
    
    print(f"서브그래프 대상: 기업 {len(top_companies)}개, 관련 스킬 {len(skills_of_top)}개")

    # 5-4. 서브그래프용 레이아웃 및 색상
    sub_company_nodes = {n for n, d in S.nodes(data=True) if d['bipartite'] == 0}
    sub_pos = nx.bipartite_layout(S, sub_company_nodes)
    sub_color_map = ['skyblue' if node in sub_company_nodes else 'lightgreen' for node in S]

    # 5-5. 서브그래프 시각화
    plt.figure(figsize=(25, 12)) # 가로로 길게
    nx.draw(S, sub_pos, 
            node_color=sub_color_map, 
            with_labels=True, 
            node_size=1500,
            font_size=10, 
            font_family=plt.rcParams['font.family'], # 설정된 한글 폰트 사용
            alpha=0.8,
            width=0.5)
            
    plt.title("상위 5개 기업-스킬 서브그래프", fontsize=25)
    plt.savefig("bipartite_subgraph.png")
    print("'bipartite_subgraph.png' 파일로 저장되었습니다.")
    plt.close()

except Exception as e:
    print(f"서브그래프 시각화 중 오류: {e}")

현재 작업 폴더: c:\socialnetwork\SocialNetwork_7
CSV 파일 로드를 시도합니다...
한글 폰트 설정 중 오류 발생 (라벨이 깨질 수 있음): 'FontEntry' object has no attribute 'path'

--- CSV 데이터 샘플 ---
             기업명     Skill
0  우아한형제들(배달의민족)       EMR
1  우아한형제들(배달의민족)  BigQuery
2  우아한형제들(배달의민족)  Planning
3  우아한형제들(배달의민족)  Superset
4  우아한형제들(배달의민족)       AWS

--- 네트워크 생성 완료 ---
총 노드 수: 2039 (기업: 1260, 스킬: 779)
총 엣지 수: 24175

[시각화 1] 전체 그래프를 그립니다 (라벨 없음)...


C:\Users\ASUS\AppData\Local\Temp\ipykernel_14108\2952546387.py:87: UserWarning: Glyph 44592 (\N{HANGUL SYLLABLE GI}) missing from font(s) DejaVu Sans.
  plt.savefig("bipartite_full_graph.png")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14108\2952546387.py:87: UserWarning: Glyph 50629 (\N{HANGUL SYLLABLE EOB}) missing from font(s) DejaVu Sans.
  plt.savefig("bipartite_full_graph.png")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14108\2952546387.py:87: UserWarning: Glyph 49828 (\N{HANGUL SYLLABLE SEU}) missing from font(s) DejaVu Sans.
  plt.savefig("bipartite_full_graph.png")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14108\2952546387.py:87: UserWarning: Glyph 53420 (\N{HANGUL SYLLABLE KIL}) missing from font(s) DejaVu Sans.
  plt.savefig("bipartite_full_graph.png")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14108\2952546387.py:87: UserWarning: Glyph 51060 (\N{HANGUL SYLLABLE I}) missing from font(s) DejaVu Sans.
  plt.savefig("bipartite_full_graph.png")
C:\Users\ASUS\AppData\Local\Temp\

'bipartite_full_graph.png' 파일로 저장되었습니다.

[시각화 2] 상위 5개 기업 서브그래프를 그립니다 (라벨 포함)...
서브그래프 대상: 기업 5개, 관련 스킬 396개


C:\Users\ASUS\AppData\Local\Temp\ipykernel_14108\2952546387.py:128: UserWarning: Glyph 49345 (\N{HANGUL SYLLABLE SANG}) missing from font(s) DejaVu Sans.
  plt.savefig("bipartite_subgraph.png")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14108\2952546387.py:128: UserWarning: Glyph 50948 (\N{HANGUL SYLLABLE WI}) missing from font(s) DejaVu Sans.
  plt.savefig("bipartite_subgraph.png")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14108\2952546387.py:128: UserWarning: Glyph 44060 (\N{HANGUL SYLLABLE GAE}) missing from font(s) DejaVu Sans.
  plt.savefig("bipartite_subgraph.png")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14108\2952546387.py:128: UserWarning: Glyph 44592 (\N{HANGUL SYLLABLE GI}) missing from font(s) DejaVu Sans.
  plt.savefig("bipartite_subgraph.png")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14108\2952546387.py:128: UserWarning: Glyph 50629 (\N{HANGUL SYLLABLE EOB}) missing from font(s) DejaVu Sans.
  plt.savefig("bipartite_subgraph.png")
C:\Users\ASUS\AppData\Local\Temp\ipy

'bipartite_subgraph.png' 파일로 저장되었습니다.
